In [1]:

from transformers import AutoTokenizer
from datasets import load_from_disk
from encoder import RobertaEncoder
from dense_retrieval import DenseRetrieval_with_Faiss

In [13]:
tokenizer = AutoTokenizer.from_pretrained('./dense_retireval_roberta_small/tokenizer')
q_encoder = RobertaEncoder.from_pretrained('./dense_retireval_roberta_small/q_encoder')
p_encoder = RobertaEncoder.from_pretrained('./dense_retireval_roberta_small/p_encoder')

In [31]:
p_encoder.cuda()
q_encoder.cuda()

RobertaEncoder(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((7

In [25]:
import torch

def embed(inputs):
  p_inputs = tokenizer(
                inputs,
                padding="max_length",
                truncation=True,
                return_tensors="pt",
              ).to('cuda')
  result = p_encoder(**p_inputs).squeeze().to("cpu").detach().numpy()
  del p_inputs
  torch.cuda.empty_cache()
  return result

In [30]:
def qued(inputs):
  q_inputs = tokenizer(
                inputs,
                padding="max_length",
                truncation=True,
                return_tensors="pt",
              ).to('cuda')
  result = q_encoder(**q_inputs).squeeze().to("cpu").detach().numpy()
  del q_inputs
  torch.cuda.empty_cache()
  return result

In [26]:
embed('이 질문은 나라목록에 대한것으로')

array([ 1.58917308e-01,  6.83112741e-01, -1.03906775e-02, -8.70562136e-01,
       -8.37832630e-01,  4.27769750e-01,  4.12514865e-01, -1.89403310e-01,
       -7.94735253e-01,  2.13853400e-02, -1.75878182e-01,  5.48718870e-01,
       -3.57209384e-01,  7.39621043e-01,  9.03636575e-01,  7.12987781e-01,
       -2.93706894e-01,  6.28493786e-01, -5.97970076e-02, -3.80331427e-01,
        8.87648106e-01, -3.30629319e-01, -2.71660592e-02,  8.00301805e-02,
       -1.46358997e-01,  7.49504149e-01,  5.61451674e-01,  5.82125068e-01,
       -3.57484698e-01, -5.95976710e-01, -9.31755126e-01,  6.94287121e-01,
        3.46760035e-01, -2.55537868e-01, -8.03604901e-01,  8.91500175e-01,
        2.04973698e-01,  7.14730859e-01,  7.52980113e-01,  1.51653633e-01,
       -5.48424721e-01, -3.45355272e-01, -9.24137712e-01,  4.70258221e-02,
        7.45471597e-01,  5.05829751e-01,  5.64749479e-01,  9.30753350e-01,
        5.28592467e-01, -3.49274635e-01, -7.95929432e-01,  1.33353829e-01,
       -4.81239587e-01, -

In [27]:
wiki_data = load_from_disk('/opt/ml/data/wiki_preprocessed_droped')

In [28]:
ds_with_embeddings = wiki_data.map(lambda example: {'embeddings': embed(example['text'])})

  0%|          | 0/55962 [00:00<?, ?ex/s]

In [33]:
ds_with_embeddings.add_faiss_index(column='embeddings')
ds_with_embeddings.save_faiss_index('embeddings', 'my_index.faiss')

wiki_data.load_faiss_index('embeddings', 'my_index.faiss')

  0%|          | 0/56 [00:00<?, ?it/s]

In [45]:
scores, retrieved_examples = wiki_data.get_nearest_examples('embeddings', embed('대한민국에서 대중적으로 인기를 얻은 짜장면과 짬뽕 등 한국식 중국 요리는 화교들이 많이 살던 인천'), k=10)

In [46]:
scores

array([2.1789410e-09, 2.2484352e-09, 2.2755759e-09, 2.5216960e-09,
       2.5558584e-09, 2.5920524e-09, 2.7212255e-09, 2.7337790e-09,
       2.7535678e-09, 2.8021192e-09], dtype=float32)

In [47]:
retrieved_examples

{'text': ['스토리는 다음과 같이 진행한다. ; 아르르 나쟈 사탄 드래코켄타우로스 스케토우다라 세죠 위그이 윗치 카방클 루루 ; 아미티 아코르 선생 라피나 리델 시그 크루크 아르르 나쟈 ; 안도우 링고 오니온 유우 & 레이 아미티 렘레스 사사키 마구로 리스쿠마 선배 에코로 ; 시그 안도우 링고 페리 ; 라피나 ; 세죠 위그이 ; 사사키 마구로 리수쿠마 선배 ; 아코르 선생 ; 리델 드라코켄타우로스 ; 사탄',
  ';세르베트 * 2000-01 스위스 컵 우승 ;바젤 * 스위스 슈퍼리그 : 2009-10, 2010-11, 2011-12, 2012-13 * 스위스 컵 우승 : 2009-10, 2011-12 * 2012-13 스위스 컵 준우승 * 2010-11 우렌컵 우승 ;개인 * 2004-05 리그 1 득점왕 * 2004-05 리그 1 베스트 11 * 2006년 FIFA 월드컵 맨 오브 더 매치 : vs. 토고, vs. 대한민국 (이상 조별리그) * 2011년 스위스 슈퍼리그 올해의 선수상 * 스위스 올해의 선수상 : 2004, 2005, 2007 * 스위스 슈퍼리그 득점왕 : 2010-11, 2011-12',
  ';스위스 그라스호퍼 * 2002-03 스위스 슈퍼리그 우승 ;이탈리아 * 2008-09 코파 이탈리아 우승 * 2009년 수페르코파 이탈리아나 우승 * 세리에 A : 우승 (2011-12, 2012-13, 2013-14, 2014-15, 2015-16, 2016-17, 2017-18) * 코파 이탈리아 : 우승 (2014-15, 2015-16, 2016-17, 2017-18) * 코파 이탈리아 : 준우승 (2011-12) * 수페르코파 이탈리아나 : 우승 (2012, 2013, 2015) * UEFA 챔피언스리그 : 준우승 (2014-15, 2016-17)',
  '; * 리그 1 : 우승 (2011-12) * FA컵 : 2014, 2015, 2017 * FA 커뮤니티 실드 : 2014, 2015 ;프랑스 * UEFA 유럽 축구 선수권 대회 